In [107]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [108]:
# Import aircraft info
aircraft_info = pd.read_excel('Aircraft_fleet_info.xlsx')

# Transpose the dataframe 
aircraft_info = aircraft_info.T

# Promote the first row to column headers and drop the first row
aircraft_info.reset_index(inplace=True)
aircraft_info.columns = aircraft_info.iloc[0]
aircraft_info = aircraft_info.drop(aircraft_info.index[0])

# Split the Aircraft Type column into two columns at : and drop the original column and rename the new columns
aircraft_info[['Aircraft Type', 'Aircraft Type Name']] = aircraft_info['Aircraft Type'].str.split(':', expand=True)
aircraft_info.set_index('Aircraft Type', inplace=True)

# Make it a dictionary for each aircraft type
aircraft_info = aircraft_info.to_dict('index')

# Add kerosen flag to the dictionary
aircraft_info['Type 1']['Electric'] = 0
aircraft_info['Type 2']['Electric'] = 1
aircraft_info['Type 3']['Electric'] = 1

# Make a list of aircraft types
aircraft_types = list(aircraft_info.keys())


In [109]:
# Import distances 
distances = pd.read_csv('Group_4Regional_Distances.csv')

# Rename the unammed column to 'Origin'
distances.rename(columns={'Unnamed: 0':'Origin'}, inplace=True)
distances.set_index('Origin', inplace=True)

# Make it a dictionary
distances = distances.to_dict()

In [129]:
# Import airport info 
airport_info = pd.read_csv('Group_4Regional_Airport_info.csv')

hub = airport_info['ICAO Code'].iloc[0]

# Make it a dictionary using the ICAO code as the key and the rest of the info in a sub-dictionary
airport_info = airport_info.set_index('ICAO Code').T.to_dict('dict')

# Create a list of the airport ICAO codes
airport_list = list(airport_info.keys())

In [134]:
# Import demand data
daily_demand = pd.read_csv('Group_4Regional_Demand.csv')
daily_demand

,Unnamed: 0,LIRF,LIPZ,LICJ,LIRQ,LIEE,LIRN,LIPX,LIBP,LIEO,LIMJ,LIPY,LIPQ
0,LIRF,0,680,390,750,275,810,595,410,290,415,415,440
1,LIPZ,680,0,320,720,260,525,845,375,240,510,495,620
2,LICJ,395,305,0,310,175,395,305,220,150,240,210,230
3,LIRQ,680,700,335,0,230,590,790,380,250,495,450,470
4,LIEE,325,230,180,270,0,245,230,140,110,190,145,160
5,LIRN,800,590,465,505,265,0,500,435,260,395,415,455
6,LIPX,650,725,350,795,245,585,0,275,220,575,385,470
7,LIBP,440,380,245,360,140,445,305,0,130,255,270,250
8,LIEO,265,220,135,215,130,235,230,135,0,180,135,155
9,LIMJ,480,435,225,505,155,405,540,235,170,0,270,340


In [111]:
# Revenue

''' Create Yield matrix dict from formula in RPK using distance matrix
    Formula: Yield = 5.9 ∙ dij^(−0.76) + 0.043 '''

yield_matrix = {}
for i in airport_list:
    yield_matrix[i] = {}
    for j in airport_list:
        if i == j:
            yield_matrix[i][j] = 0
        else:
            yield_matrix[i][j] = 5.9 * (distances[i][j] ** (-0.76)) + 0.043


In [131]:
# Costs 
# Operational costs

''' Fixed costs = CXk  '''
fixed = {}
for k in aircraft_types:
    fixed[k] = aircraft_info[k]['Fixed Operating Cost (Per Fligth Leg)  [€]']

''' Time based costs i -> j
    Formula: CTkij = cTk ∙ dij / Vk '''
time_based = {}
for k in aircraft_types:
    time_based[k] = {}
    for i in airport_list:
        time_based[k][i] = {}
        for j in airport_list:
            time_based[k][i][j] = aircraft_info[k]['Cost per Hour'] * distances[i][j] / aircraft_info[k]['Speed [km/h]']

''' Fuel costs i -> j
    Formula: CFkij = cFk ∙ dij ∙ f / 1.5'''
fuel_gal_cost = 1.42 
fuel = {}
for k in aircraft_types:
    fuel[k] = {}
    for i in airport_list:
        fuel[k][i] = {}
        for j in airport_list:
            if aircraft_info[k]['Electric'] == 1:
                fuel[k][i][j] = 0
            else:
                fuel[k][i][j] = aircraft_info[k]['Fuel Cost Parameter'] * distances[i][j] * fuel_gal_cost / 1.5

''' Energy costs i -> j
    Formula: CEkij = e ∙ Gk ∙ dij / Rk '''
energy = {}
for k in aircraft_types:
    energy[k] = {}
    for i in airport_list:
        energy[k][i] = {}
        for j in airport_list:
            if aircraft_info[k]['Electric'] == 0:
                energy[k][i][j] = 0
            else:
                energy[k][i][j] = 0.07 * aircraft_info[k]['Batteries energy (kWh)'] * distances[i][j] / aircraft_info[k]['Maximum Range [km]']

''' Total costs i -> j
    Formula: CXk + CTkij + CFkij + CEkij '''
hub_factor = 0.7
op_costs = {}
for k in aircraft_types:
    op_costs[k] = {}
    for i in airport_list:
        op_costs[k][i] = {}
        for j in airport_list:
            if i == hub or j == hub:
                op_costs[k][i][j] = (fixed[k] + time_based[k][i][j] + fuel[k][i][j]) * hub_factor + energy[k][i][j]
            else:
                op_costs[k][i][j] = fixed[k] + time_based[k][i][j] + fuel[k][i][j] + energy[k][i][j]
                

In [ ]:

# Create a dictionary with the aircraft types as keys and the rest of the columns as sub-dictionaries
aircrafts = {}
for index, row in aircraft_info.iterrows():
    aircrafts[row['AC_type']] = row.to_dict()

# Rename the main keys in the aircraft dictionary to a numeric value with format AC_#

# Create a list of the keys in the aircraft dictionary
keys = list(aircrafts.keys())

# Create a new dictionary with the new keys
new_keys = {}
for i in range(len(keys)):
    new_keys[keys[i]] = 'AC_' + str(i+1)

aircraft_types = list(new_keys.values())

# Create a new dictionary with the new keys
aircrafts_new = {}
for i in range(len(keys)):
    aircrafts_new['AC_' + str(i+1)] = aircrafts[keys[i]]

# Replace the keys in the aircraft dictionary with the new keys
aircrafts = aircrafts_new
